In [1]:
%pip install groq
%pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from groq import Groq
from dotenv import load_dotenv
import re
import os

load_dotenv()
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

Here we have some prompts for testing

In [3]:
order_item_prompt = """
I will give you a JavaEE file for which I want to take one step towards migrating to Quarkus.

I will provide you with static source code analysis information highlighting an issue which needs to be addressed.

I will also provide you with an example of how a similar issue was solved in the past via a solved example.

You can refer to the solved example for a pattern of how to update the input Java EE file to Quarkus.

Fix only the problem described. Other problems will be solved in subsequent steps so it is unnecessary to handle them now.

Before attempting to migrate the code to Quarkus reason through what changes are required and why.

Pay attention to changes you make and impacts to external dependencies in the pom.xml as well as changes to imports we need to consider.

Remember when updating or adding annotations that the class must be imported.

As you make changes that impact the pom.xml or imports, be sure you explain what needs to be updated.

After you have shared your step by step thinking, provide a full output of the updated file.

# Input information

## Input File

File name: "src/main/java/com/redhat/coolstore/model/OrderItem.java"
Source file contents:
```java
package com.redhat.coolstore.model;

import java.io.Serializable;

import javax.persistence.Column;
import javax.persistence.Entity;
import javax.persistence.GeneratedValue;
import javax.persistence.Id;
import javax.persistence.Table;

@Entity
@Table(name = "ORDER_ITEMS")
public class OrderItem implements Serializable {
	private static final long serialVersionUID = 64565445665456666L;

	@Id
	@Column(name="ID")
	@GeneratedValue
	private long id;

	private int quantity;

	private String productId;

	public OrderItem() {}

	public String getProductId() {
		return productId;
	}

	public void setProductId(String productId) {
		this.productId = productId;
	}

	public int getQuantity() {
		return quantity;
	}

	public void setQuantity(int quantity) {
		this.quantity = quantity;
	}

	@Override
	public String toString() {
		return "OrderItem [productId=" + productId + ", quantity=" + quantity + "]";
	}

}
```

## Issues

### incident 0
incident to fix: "The way in which Hibernate determines implicit names for sequences and tables associated with identifier generation has changed in 6.0 which may affect migrating applications. 
 As of 6.0, Hibernate by default creates a sequence per entity hierarchy instead of a single sequence hibernate_sequence. 
 Due to this change, users that previously used `@GeneratedValue(strategy = GenerationStrategy.AUTO)` or simply `@GeneratedValue` (since `AUTO` is the default), need to ensure that the database now contains sequences for every entity, named `<entity name>_seq`. For an entity Person, a sequence person_seq is expected to exist. 
 It’s best to run hbm2ddl (e.g. by temporarily setting `hbm2ddl.auto=create`) to obtain a list of DDL statements for the sequences."
Line number: 18
### incident 1
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 5
### incident 2
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 6
### incident 3
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 7
### incident 4
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 8
### incident 5
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 9

# Output Instructions
Structure your output in Markdown format such as:

## Reasoning
Write the step by step reasoning in this markdown section. If you are unsure of a step or reasoning, clearly state you are unsure and why.

## Updated File
```java
// Write the updated file for Quarkus in this section. If the file should be removed, make the content of the updated file a comment explaining it should be removed.
```

## Additional Information (optional)

If you have any additional details or steps that need to be performed, put it here.
"""

In [4]:
inventory_entity_MDB_prompt = """
I will give you a JavaEE file for which I want to take one step towards migrating to Quarkus.

I will provide you with static source code analysis information highlighting an issue which needs to be addressed.

I will also provide you with an example of how a similar issue was solved in the past via a solved example.

You can refer to the solved example for a pattern of how to update the input Java EE file to Quarkus.

Fix only the problem described. Other problems will be solved in subsequent steps so it is unnecessary to handle them now.

Before attempting to migrate the code to Quarkus reason through what changes are required and why.

Pay attention to changes you make and impacts to external dependencies in the pom.xml as well as changes to imports we need to consider.

Remember when updating or adding annotations that the class must be imported.

As you make changes that impact the pom.xml or imports, be sure you explain what needs to be updated.

After you have shared your step by step thinking, provide a full output of the updated file.

# Input information

## Input File

File name: "src/main/java/com/redhat/coolstore/service/InventoryNotificationMDB.java"
Source file contents:
```java
package com.redhat.coolstore.service;

import com.redhat.coolstore.model.Order;
import com.redhat.coolstore.utils.Transformers;

import javax.inject.Inject;
import javax.jms.*;
import javax.naming.Context;
import javax.naming.InitialContext;
import javax.naming.NamingException;
import javax.rmi.PortableRemoteObject;
import java.util.Hashtable;

public class InventoryNotificationMDB implements MessageListener {

    private static final int LOW_THRESHOLD = 50;

    @Inject
    private CatalogService catalogService;

    private final static String JNDI_FACTORY = "weblogic.jndi.WLInitialContextFactory";
    private final static String JMS_FACTORY = "TCF";
    private final static String TOPIC = "topic/orders";
    private TopicConnection tcon;
    private TopicSession tsession;
    private TopicSubscriber tsubscriber;

    public void onMessage(Message rcvMessage) {
        TextMessage msg;
        {
            try {
                System.out.println("received message inventory");
                if (rcvMessage instanceof TextMessage) {
                    msg = (TextMessage) rcvMessage;
                    String orderStr = msg.getBody(String.class);
                    Order order = Transformers.jsonToOrder(orderStr);
                    order.getItemList().forEach(orderItem -> {
                        int old_quantity = catalogService.getCatalogItemById(orderItem.getProductId()).getInventory().getQuantity();
                        int new_quantity = old_quantity - orderItem.getQuantity();
                        if (new_quantity < LOW_THRESHOLD) {
                            System.out.println("Inventory for item " + orderItem.getProductId() + " is below threshold (" + LOW_THRESHOLD + "), contact supplier!");
                        } else {
                            orderItem.setQuantity(new_quantity);
                        }
                    });
                }


            } catch (JMSException jmse) {
                System.err.println("An exception occurred: " + jmse.getMessage());
            }
        }
    }

    public void init() throws NamingException, JMSException {
        Context ctx = getInitialContext();
        TopicConnectionFactory tconFactory = (TopicConnectionFactory) PortableRemoteObject.narrow(ctx.lookup(JMS_FACTORY), TopicConnectionFactory.class);
        tcon = tconFactory.createTopicConnection();
        tsession = tcon.createTopicSession(false, Session.AUTO_ACKNOWLEDGE);
        Topic topic = (Topic) PortableRemoteObject.narrow(ctx.lookup(TOPIC), Topic.class);
        tsubscriber = tsession.createSubscriber(topic);
        tsubscriber.setMessageListener(this);
        tcon.start();
    }

    public void close() throws JMSException {
        tsubscriber.close();
        tsession.close();
        tcon.close();
    }

    private static InitialContext getInitialContext() throws NamingException {
        Hashtable<String, String> env = new Hashtable<>();
        env.put(Context.INITIAL_CONTEXT_FACTORY, JNDI_FACTORY);
        env.put(Context.PROVIDER_URL, "t3://localhost:7001");
        env.put("weblogic.jndi.createIntermediateContexts", "true");
        return new InitialContext(env);
    }
}
```

## Issues

### incident 0
incident to fix: "Replace the `javax.inject` import statement with `jakarta.inject`"
Line number: 6
### incident 1
incident to fix: "References to JavaEE/JakartaEE JMS elements should be removed and replaced with their Quarkus SmallRye/Microprofile equivalents."
Line number: 7

# Output Instructions
Structure your output in Markdown format such as:

## Reasoning
Write the step by step reasoning in this markdown section. If you are unsure of a step or reasoning, clearly state you are unsure and why.

## Updated File
```java
// Write the updated file for Quarkus in this section. If the file should be removed, make the content of the updated file a comment explaining it should be removed.
```

## Additional Information (optional)

If you have any additional details or steps that need to be performed, put it here.

"""

In [6]:
shopping_cart_services_prompt = """
I will give you a JavaEE file for which I want to take one step towards migrating to Quarkus.

I will provide you with static source code analysis information highlighting an issue which needs to be addressed.

I will also provide you with an example of how a similar issue was solved in the past via a solved example.

You can refer to the solved example for a pattern of how to update the input Java EE file to Quarkus.

Fix only the problem described. Other problems will be solved in subsequent steps so it is unnecessary to handle them now.

Before attempting to migrate the code to Quarkus reason through what changes are required and why.

Pay attention to changes you make and impacts to external dependencies in the pom.xml as well as changes to imports we need to consider.

Remember when updating or adding annotations that the class must be imported.

As you make changes that impact the pom.xml or imports, be sure you explain what needs to be updated.

After you have shared your step by step thinking, provide a full output of the updated file.

# Input information

## Input File

File name: "src/main/java/com/redhat/coolstore/service/ShoppingCartService.java"
Source file contents:
```java
package com.redhat.coolstore.service;

import java.util.Hashtable;
import java.util.logging.Logger;

import javax.ejb.Stateful;
import javax.inject.Inject;
import javax.naming.Context;
import javax.naming.InitialContext;
import javax.naming.NamingException;

import com.redhat.coolstore.model.Product;
import com.redhat.coolstore.model.ShoppingCart;
import com.redhat.coolstore.model.ShoppingCartItem;

@Stateful
public class ShoppingCartService  {

    @Inject
    Logger log;

    @Inject
    ProductService productServices;

    @Inject
    PromoService ps;


    @Inject
    ShoppingCartOrderProcessor shoppingCartOrderProcessor;

    private ShoppingCart cart  = new ShoppingCart(); //Each user can have multiple shopping carts (tabbed browsing)

   

    public ShoppingCartService() {
    }

    public ShoppingCart getShoppingCart(String cartId) {
        return cart;
    }

    public ShoppingCart checkOutShoppingCart(String cartId) {
        ShoppingCart cart = this.getShoppingCart(cartId);
      
        log.info("Sending  order: ");
        shoppingCartOrderProcessor.process(cart);
   
        cart.resetShoppingCartItemList();
        priceShoppingCart(cart);
        return cart;
    }

    public void priceShoppingCart(ShoppingCart sc) {

        if (sc != null) {

            initShoppingCartForPricing(sc);

            if (sc.getShoppingCartItemList() != null && sc.getShoppingCartItemList().size() > 0) {

                ps.applyCartItemPromotions(sc);

                for (ShoppingCartItem sci : sc.getShoppingCartItemList()) {

                    sc.setCartItemPromoSavings(
                            sc.getCartItemPromoSavings() + sci.getPromoSavings() * sci.getQuantity());
                    sc.setCartItemTotal(sc.getCartItemTotal() + sci.getPrice() * sci.getQuantity());

                }

                sc.setShippingTotal(lookupShippingServiceRemote().calculateShipping(sc));

                if (sc.getCartItemTotal() >= 25) {
                    sc.setShippingTotal(sc.getShippingTotal()
                            + lookupShippingServiceRemote().calculateShippingInsurance(sc));
                }

            }

            ps.applyShippingPromotions(sc);

            sc.setCartTotal(sc.getCartItemTotal() + sc.getShippingTotal());

        }

    }

    private void initShoppingCartForPricing(ShoppingCart sc) {

        sc.setCartItemTotal(0);
        sc.setCartItemPromoSavings(0);
        sc.setShippingTotal(0);
        sc.setShippingPromoSavings(0);
        sc.setCartTotal(0);

        for (ShoppingCartItem sci : sc.getShoppingCartItemList()) {
            Product p = getProduct(sci.getProduct().getItemId());
            //if product exist
            if (p != null) {
                sci.setProduct(p);
                sci.setPrice(p.getPrice());
            }

            sci.setPromoSavings(0);
        }

    }

    public Product getProduct(String itemId) {
        return productServices.getProductByItemId(itemId);
    }

	private static ShippingServiceRemote lookupShippingServiceRemote() {
        try {
            final Hashtable<String, String> jndiProperties = new Hashtable<>();
            jndiProperties.put(Context.INITIAL_CONTEXT_FACTORY, "org.wildfly.naming.client.WildFlyInitialContextFactory");

            final Context context = new InitialContext(jndiProperties);

            return (ShippingServiceRemote) context.lookup("ejb:/ROOT/ShippingService!" + ShippingServiceRemote.class.getName());
        } catch (NamingException e) {
            throw new RuntimeException(e);
        }
    }
}

```

## Issues

### incident 0
incident to fix: "Replace the `javax.ejb` import statement with `jakarta.ejb`"
Line number: 6
Solved example diff:
```diff


### incident 1
incident to fix: "Replace the `javax.inject` import statement with `jakarta.inject`"
Line number: 7
Solved example diff:
```diff


### incident 2
incident to fix: "Stateful EJBs can be converted to a CDI bean by replacing the `@Stateful` annotation with a bean-defining annotation
 that encompasses the appropriate scope (e.g., `@ApplicationScoped`). `@Stateful` EJBs often translate to `@SessionScoped`
 beans (a scope which requires activating the `quarkus-undertow` extension), but the appropriate scope may differ based
 on your application architecture. Review your application's requirements to determine the appropriate scope.

 Note that it is recommended, as a good practice, to keep state external from the service in Quarkus."
Line number: 16
Solved example diff:
```diff



# Output Instructions
Structure your output in Markdown format such as:

## Reasoning
Write the step by step reasoning in this markdown section. If you are unsure of a step or reasoning, clearly state you are unsure and why.

## Updated File
```java
// Write the updated file for Quarkus in this section. If the file should be removed, make the content of the updated file a comment explaining it should be removed.
```

## Additional Information (optional)

If you have any additional details or steps that need to be performed, put it here.

"""

This prompt below is know for its incomplete results during the ```run_demo.py```.

In [5]:
inventory_entity_prompt = """
I will give you a JavaEE file for which I want to take one step towards migrating to Quarkus.

I will provide you with static source code analysis information highlighting an issue which needs to be addressed.

I will also provide you with an example of how a similar issue was solved in the past via a solved example.

You can refer to the solved example for a pattern of how to update the input Java EE file to Quarkus.

Fix only the problem described. Other problems will be solved in subsequent steps so it is unnecessary to handle them now.

Before attempting to migrate the code to Quarkus reason through what changes are required and why.

Pay attention to changes you make and impacts to external dependencies in the pom.xml as well as changes to imports we need to consider.

Remember when updating or adding annotations that the class must be imported.

As you make changes that impact the pom.xml or imports, be sure you explain what needs to be updated.

After you have shared your step by step thinking, provide a full output of the updated file.

# Input information

## Input File

File name: "src/main/java/com/redhat/coolstore/model/InventoryEntity.java"
Source file contents:
```java
package com.redhat.coolstore.model;

import java.io.Serializable;

import javax.persistence.Column;
import javax.persistence.Entity;
import javax.persistence.Id;
import javax.persistence.Table;
import javax.persistence.UniqueConstraint;
import javax.xml.bind.annotation.XmlRootElement;

@Entity
@XmlRootElement
@Table(name = "INVENTORY", uniqueConstraints = @UniqueConstraint(columnNames = "itemId"))
public class InventoryEntity implements Serializable {

	private static final long serialVersionUID = 7526472295622776147L; 

    @Id
    private String itemId;


    @Column
    private String location;


    @Column
    private int quantity;


    @Column
    private String link;

    public InventoryEntity() {

    }

    public String getItemId() {
		return itemId;
	}

	public void setItemId(String itemId) {
		this.itemId = itemId;
	}

	public String getLocation() {
		return location;
	}

	public void setLocation(String location) {
		this.location = location;
	}

	public int getQuantity() {
		return quantity;
	}

	public void setQuantity(int quantity) {
		this.quantity = quantity;
	}

	public String getLink() {
		return link;
	}

	public void setLink(String link) {
		this.link = link;
	}

	@Override
    public String toString() {
        return "InventoryEntity [itemId=" + itemId + ", availability=" + quantity + "/" + location + " link=" + link + "]";
    }
}

```

## Issues

### incident 0
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 5
### incident 1
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 6
### incident 2
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 7
### incident 3
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 8
### incident 4
incident to fix: "Replace the `javax.persistence` import statement with `jakarta.persistence`"
Line number: 9
### incident 5
incident to fix: "Replace the `javax.xml` import statement with `jakarta.xml`"
Line number: 10

# Output Instructions
Structure your output in Markdown format such as:

## Reasoning
Write the step by step reasoning in this markdown section. If you are unsure of a step or reasoning, clearly state you are unsure and why.

## Updated File
```java
// Write the updated file for Quarkus in this section. If the file should be removed, make the content of the updated file a comment explaining it should be removed.
```

## Additional Information (optional)

If you have any additional details or steps that need to be performed, put it here.
"""

In [7]:
def separate_sections(document):
    section_titles = ["## Reasoning", "## Updated File", "## Additional Information"]
    # Find the start index of each section by looking for the section titles, filter out not found (-1) indices
    indices = {
        title: document.find(title)
        for title in section_titles
        if document.find(title) != -1
    }

    sorted_indices = dict(sorted(indices.items(), key=lambda item: item[1]))
    sorted_indices["end"] = len(document)
    titles_sorted = list(sorted_indices.keys()) + ["end"]

    # Extract the sections based on the indices found
    sections = {}
    for i, title in enumerate(
        titles_sorted[:-1]
    ):  # Skip the last item ('end') for iteration
        start_index = sorted_indices[title] + len(title)
        end_title = titles_sorted[i + 1]
        end_index = sorted_indices[end_title]
        sections[title] = document[start_index:end_index].strip()

    return sections

In [11]:
def parse_file_solution_content(content: str):
    code_block_pattern = r"```(?:\w+)?\s+(.+?)```"

    sections = separate_sections(content)
    reasoning = sections.get("## Reasoning", "")
    updated_file_content = sections.get("## Updated File", "")
    additional_info = sections.get("## Additional Information", "")

    code_block_matches = re.findall(code_block_pattern, updated_file_content, re.DOTALL)
    matching_blocks = []
    for block in code_block_matches:
        matching_blocks.append(block)

    if matching_blocks:
        # If multiple matches default to first
        updated_file = matching_blocks[0].strip()
        if len(matching_blocks) > 1:
            return(f"Multiple matching codeblocks found, defaulting to first")
        else:
            return(f"Found single matching codeblock")
    elif code_block_matches:
        # fallback to first discovered codeblock
        updated_file = code_block_matches[0]
        if len(code_block_matches) > 1:
            return(f"Multiple codeblocks found, defaulting to first")
        else:
            return(f"Found single codeblock")
    else:
        updated_file = ""
        return("No codeblocks detected in LLM response")

In [12]:
def ensure_codeblock_in_response(prompt: str):
    messages = [
        {
            "role": "system",
            "content": "You are an AI Assistant trained on migrating enterprise JavaEE code to Quarkus.",
        },
        {"role": "user", "content": prompt},
    ]

    is_codeblock_error = True

    while is_codeblock_error:
        completion = client.chat.completions.create(
            model="mixtral-8x7b-32768",
            messages=messages,
            temperature=0,
            max_tokens=4096,
            top_p=1,
            stream=False,
            stop=None,
        )

        response = completion.choices[0].message.content
        code_block_message = parse_file_solution_content(response)

        if code_block_message == "No codeblocks detected in LLM response":

            messages.append(
                {
                    "role": "user",
                    "content": "Please provide a complete response.",
                }
            )
            print(response)
        else:

            is_codeblock_error = False
            print(response)

As you can see, the first response here was incomplete, as it did not close the response with three backticks. However, when we resubmitted the request to the LLM, we received a complete response.

In [13]:
(ensure_codeblock_in_response(inventory_entity_prompt))

## Reasoning

The incidents reported all relate to import statements that need to be updated to use the Jakarta EE namespace instead of the Java EE namespace. This is necessary because Quarkus uses the Jakarta EE APIs, not the Java EE APIs.

1. Replace the `javax.persistence` import statement with `jakarta.persistence` on lines 5, 6, 7, 8, and 9. This is necessary because Quarkus uses the Jakarta Persistence API, which is part of Jakarta EE, not Java EE. The `javax.persistence` package is deprecated in Jakarta EE and has been replaced with `jakarta.persistence`.

2. Replace the `javax.xml` import statement with `jakarta.xml` on line 10. This is necessary because Quarkus uses the Jakarta XML Binding API, which is part of Jakarta EE, not Java EE. The `javax.xml` package is deprecated in Jakarta EE and has been replaced with `jakarta.xml`.

## Updated File
```java
package com.redhat.coolstore.model;

import java.io.Serializable;

import jakarta.persistence.Column;
import jakarta.persisten

In [14]:
print(ensure_codeblock_in_response(shopping_cart_services_prompt))

## Reasoning

1. **Incident 0:** The `javax.ejb` package is part of the Java EE specification, while Quarkus uses the Jakarta EE specification. Therefore, we need to replace the `javax.ejb` import statement with `jakarta.ejb`.

2. **Incident 1:** Similar to incident 0, we need to replace the `javax.inject` import statement with `jakarta.inject` as Quarkus uses the Jakarta EE specification.

3. **Incident 2:** Stateful EJBs are not natively supported in Quarkus. Instead, we can use CDI beans with a suitable scope. In this case, we will use `@ApplicationScoped` as a starting point, but it's recommended to keep state external from the service in Quarkus. We will remove the `@Stateful` annotation and replace it with `@ApplicationScoped`.

## Updated File

```java
package com.redhat.coolstore.service;

import java.util.Hashtable;
import java.util.logging.Logger;

import jakarta.ejb.Stateful; // Changed to jakarta.ejb
import jakarta.inject.ApplicationScoped; // Added @ApplicationScoped
impor

In [15]:
print(ensure_codeblock_in_response(order_item_prompt))

## Reasoning

**incident 1, 2, 3, 4, 5:**

The import statements are using the `javax.persistence` package, which is the Java EE package for JPA. In Quarkus, we should use the `jakarta.persistence` package instead, which is the Jakarta EE package for JPA. This change is required because Java EE has been renamed to Jakarta EE, and the package names have changed accordingly.

**incident 0:**

The `@GeneratedValue` annotation is used to generate primary keys for the `OrderItem` entity. In Hibernate 6.0, the default strategy for generating primary keys has changed, and it now creates a sequence per entity hierarchy instead of a single sequence `hibernate_sequence`. This means that for each entity, a sequence with the entity name followed by `_seq` is expected to exist in the database.

In this case, we are using the default strategy (`@GeneratedValue` without any parameters), so we need to ensure that a sequence named `order_items_seq` exists in the database. We can use the Hibernate tool 

In [16]:
print(ensure_codeblock_in_response(inventory_entity_MDB_prompt))

## Reasoning

**Incident 0:**

The `javax.inject` package is from the Java EE specification, and it should be replaced with `jakarta.inject` for Quarkus as Quarkus follows the Jakarta EE specification.

**Incident 1:**

Java EE/Jakarta EE JMS elements like `javax.jms.*` should be replaced with their Quarkus SmallRye/Microprofile equivalents. Quarkus uses SmallRye as the implementation for Jakarta EE APIs, and it provides an alternative for JMS called `io.smallrye.common.annotation.Blocking`.

## Updated File

```java
package com.redhat.coolstore.service;

import com.redhat.coolstore.model.Order;
import com.redhat.coolstore.utils.Transformers;
import io.smallrye.common.annotation.Blocking;
import jakarta.inject.Inject;

import java.util.Hashtable;

public class InventoryNotificationMDB implements MessageListener {

    private static final int LOW_THRESHOLD = 50;

    @Inject
    private CatalogService catalogService;

    @Blocking
    private TopicConnection tcon;
    @Blocking
    pr